<a href="https://colab.research.google.com/github/GeorgeFane/LyricsFreak-Scraper/blob/main/LyricsFreak_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

I'm working with some friends on a lyric analyzer (trying not to be too specific). It will be pretty ambitious with a lot of web scraping and data analysis, so we're building it piece by piece. I started with the web scraping side of things because I have experience from building the MDining Scraper.  The app on this page will be a small part of our project.

The idea is to start with an artist name and count the frequency of all words in all their songs. Breaking it down further, we pick a site that lists artists and stores lyrics, go to an artist's page, get links to all their songs' lyrics, follow those links to each song's page, scrape the lyrics off that page, combine all songs' lyrics into a list or string, and turn that into a frequency dictionary.

# Code

## Required Libraries 

Requests for getting a link's content, asyncio for requesting multiple links concurrently, nest_asyncio to prevent async errors, lxml for parsing that content to an HTML object, re for finding words in a string through regex, collections.Counter for quickly making frequency dictionaries, datetime for timing performance, json for dumping data to files, and pandas for displaying data.

In [ ]:
import requests
import asyncio
import nest_asyncio
from lxml.html import fromstring, tostring
import re
from collections import Counter
from datetime import datetime as dt
import json
import pandas as pd

## Links for Artist and Songs

Takes a name and creates the LyricFreak link. For the example of Hans Zimmer, the link is https://www.lyricsfreak.com/h/hans+zimmer/

In [ ]:
name = 'hans zimmer'
start = dt.now()

single = name.lower().replace(' ', '+')
site = 'https://www.lyricsfreak.com'
link = f"{site}/{single[0]}/{single}/"

Gets all song links for an artist. Looking at the XPath query, we first look find divs with class="lf-list js-sort-table". We do this because there are multiple divs on the page, and only this one contains songs with the artist as primary credit. Another div contains additional songs, but those are just features and we don't want to get those.

Inside the div we want, we get the href tags for each a component, because each is a song. We store all the song links in 'songs'.

In [ ]:
r = requests.get(link)
tree = fromstring(r.content)

path = '//div[@class="lf-list js-sort-table"]//a[@class="lf-link lf-link--primary"]/@href'
songs = tree.xpath(path)

## Asynchronous Requests

I began with a straightforward for loop to request.get all song pages. However, doing things sequentially took way too long, maybe half a second per page. I knew I had to start learning about asynchronous programming.

The first line I have in this code because it prevents errors. I saw an error, searched StackOverflow, and copied that line of code, and the error stopped occurring. I don't get it but I don't question it.

I then create future tasks, each of which requests a song page's content. I gather all future responses, then dump all of them to 'resps.txt'. I dump 'resps' because I can't access variables inside an async function. Any inits or assignments seem to be completely local.  

I could've used grequests or aiohttp for this, but I couldn't figure out how to work them. With just asyncio and requests, though, I nicely don't have to pip install any additional libraries.

In [ ]:
nest_asyncio.apply()
async def main(loop):
    futures = [loop.run_in_executor(None, requests.get, site + song) for song in songs]
    resps = await asyncio.gather(*futures)
    with open('/content/resps.txt', 'w') as f:
        json.dump(
            [resp.content.decode('utf8') for resp in resps], 
            f, indent=4
        )

loop = asyncio.get_event_loop()
loop.run_until_complete(main(loop))

## Getting All Lyrics

I retrieve all pages' contents with json.load and convert them into HTML objects with fromstring. I then get every line of lyrics for every song and combine them into a flat list of strings.

In [ ]:
with open('/content/resps.txt', 'r') as f:
    trees = [fromstring(content) for content in json.load(f)]

path = '//div[@class="lyrictxt js-lyrics js-share-text-content"]/text()'
lines = [line for tree in trees for line in tree.xpath(path)]

Then, I join all lines into a single string and use a regex to find all words, including those with apostrophes and dashes. I turn that list of words into a frequency dictionary with Counter and display it in a DataFrame.

In [ ]:
whole = ' '.join(lines)
words = re.findall("[\w'-]+", whole.lower())
c = Counter(words)

print('Elapsed:', dt.now() - start)
print(len(c), 'different words')
if len(c) == 0:
    print('Probably no artist with that name')
print()

%load_ext google.colab.data_table
pd.DataFrame({
    'Word': c.keys(),
    'Count': c.values(),
})

# Conclusion

I'm delighted to work with web scraping again, and the final project will be quite cool once you see it.

---
Thank you for reading!

George Fane

With Alex Beloiu and Yongwei Che